In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.edge.options import Options ### CONFIGURADO CON EDGE
import pandas as pd
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
#import cv2
#import pytesseract
import base64
import numpy as np
import os
import random
import re

import selenium
selenium.__version__

'4.8.2'

In [2]:
iter = 1
def create_folders(iter):
    print("\n\nRevisando existencia de carpetas necesarias")
    dir_path = os.path.dirname(os.path.realpath("__file__"))
    print("path local: ", dir_path)
    folderpath = "Trovit"
    folderpath = os.path.join(dir_path, folderpath)
    if not os.path.exists(folderpath):
        os.mkdir(folderpath)
        print("Carpeta Parairnos creada")
    else: print("Carpeta Parairnos existente")
    dir_path = folderpath
    folderpath = f"Data_porpag_{iter}"
    folderpath = os.path.join(dir_path, folderpath)
    if not os.path.exists(folderpath):
        os.mkdir(folderpath)
        print("Subcarpeta Data creada\n\n")
    else: print("Subcarpeta Data Existente")
    return folderpath, dir_path

folderpath, dir_path = create_folders(1)



Revisando existencia de carpetas necesarias
path local:  C:\Users\bigdatauls\Documents\workspace\python\Webscrapper_arriendos
Carpeta Parairnos existente
Subcarpeta Data Existente


In [3]:
options = Options()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument("--disable-notifications")

def opendriver(options):
    driver = webdriver.Edge(options=options)
    time.sleep(2)
    driver.maximize_window()
    print("Abriendo WebDriver")
    time.sleep(2)
    return driver

driver = opendriver(options)

Abriendo WebDriver


In [4]:
busqueda = "Coquimbo"
url = "https://casas.trovit.cl/index.php/cod.search_homes/type.2/what_d.Coquimbo%20Region/sug.0/isUserSearch.1/origin.2/order_by.relevance/region.Coquimbo/"
def apertura_portal(driver, busqueda, url):
    driver.get(url)
    time.sleep(3)
    print("Cargando Página y realizando búsqueda de '", busqueda, "'")
    #buscador = driver.find_element(By.XPATH,'//*[@id="parairnosSearch"]')
    #buscador.click()
    #buscador.send_keys(Keys.CONTROL, "a")
    #buscador.send_keys(Keys.DELETE)
    time.sleep(1)
    
    #buscador = driver.find_el ement(By.XPATH, '//input[@id="bigsearch-query-location-input"]')
    #buscador.clear()   ]
    
    #buscador.send_keys(busqueda)
    #button = driver.find_elements(By.XPATH,'//button[@type="submit"]')
    #button[1].click()
    time.sleep(3)
apertura_portal(driver, busqueda, url)

Cargando Página y realizando búsqueda de ' Coquimbo '


In [5]:
df = pd.DataFrame()
#link = "https://casas.trovit.cl/listing/marina-oceano-departamento-temporada-coquimbo.5b669189-f556-37de-8e29-55b78291c378"
#link = "https://casas.trovit.cl/listing/departamento-2d2b1estac-amoblado-marzo-a-diciembre-coquimbo.f3216ba0-adab-11ed-bdbb-2143f1ac185d"
#link = "https://casas.trovit.cl/listing/arriendo-marzo-a-diciembre-3d2b-estac-wiffi-coquimbo.d25bdb20-aef1-11ed-a9ea-198c56376a6d"
#link = "https://casas.trovit.cl/listing/arriendo-diario-por-temporada-de-verano-en-la-serena-3-d-1-b-1-est-con-wiffi-a-pasos-de-la-playa.2f368e82-4aa5-4620-b2e1-8135bedfd0f6"
link = "https://casas.trovit.cl/listing/departamento-en-arriendo-con-vista-despejada.9b9f9f4d-0e64-3810-b074-27442f6d82bd"
link = "https://casas.trovit.cl/listing/arriendo-departamento-amoblado-penuelas-coquimbo.ac7a5d6f-2609-4038-9e7f-f0bbe64d7686"
link = "https://casas.trovit.cl/listing/arriendo-amoblado-palmas-de-penuelas-coquimbo.aab5dfeb-6749-428f-aba6-151fd337b5b0"
dfpage = pd.DataFrame()

def scrap_trovit(driver, link, df, dfpage):
    #driver.get(link)
    time.sleep(4)
    try:
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="map"]')))
    except (TimeoutException, NoSuchElementException):
        print("Timed out waiting for element")
    #time.sleep(3)
    try:
        googlemap = driver.find_element(By.XPATH, '//*[@id="map"]')
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", googlemap)
    except (TimeoutException, NoSuchElementException):
        print("Timed out waiting for element")
    #time.sleep(4)
    #wait = WebDriverWait(driver, 10)
    #wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    link = driver.current_url

    Link = {"Link":link}
    print(Link)
    info_comuna = driver.find_element(By.XPATH, '//*[@id="main_info"]/*[@class="address"]').text
    comuna = info_comuna.split(", ")

    if len(comuna) > 3:
        Comuna = {"Comuna": comuna[1]}
    else:   
        Comuna = {"Comuna": comuna[0]}
    print(Comuna)

    Region = {"Region": comuna[-1]}
    print(Region)

    try: Baños = {"Baños": ''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//div[contains(text(), "Baño")]/span[1]').text))}
    except: Baños = {"Baños": None}
    print(Baños)

    #Capacidad = {"Capacidad": max_number}
    #print(Capacidad)
    
    try: Habitaciones = {"Habitaciones": ''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//div[contains(text(), "Dorm")]').text))}
    except: Habitaciones = {"Habitaciones": None}
    print(Habitaciones)
    
    try:
        infoprecio = ''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//div[@class="price"]//*[@class="amount"]').text))
        if infoprecio == "":    
            {"Precio": None} 
        else: Precio = {"Precio": infoprecio}
    except: Precio = {"Precio": None}
    print(Precio)

    if "$" in driver.find_element(By.XPATH, '//div[@class="price"]//*[@class="amount"]').text:
        Tipo_precio = {"Tipo_precio": "CLP"}
    elif "UF" in driver.find_element(By.XPATH, '//div[@class="price"]//*[@class="amount"]').text:
        Tipo_precio = {"Tipo_precio": "UF"}
    else: Tipo_precio = {"Tipo_precio": None}
    print(Tipo_precio)
    #try: Nota = {"Nota":driver.find_element(By.XPATH, '//*[@id="userComments"]/div[1]/div/div/div[2]/div[1]').text}
    #except: Nota = {"Nota": None}
    #print(Nota)

    #try: Nnota = {"Nnota":''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//*[@id="userComments"]/div[1]/div/div/div[2]/div[3]').text))}
    #except: Nnota = {"Nota": None}
    #print(Nnota)

    #try: 
    #    info_comentarios = driver.find_elements(By.XPATH, '//*[@id="userComments"]/div[2]/div/div/div/div/p')
    #    comentarios = []
    #    for item in info_comentarios:
    #        comentarios.append(item.text)
    #    Comentarios = {"Comentarios": comentarios}
    #except: {"Comentarios": None}
    #print(Comentarios)

    try: Descripcion = {"Descripción":driver.find_element(By.XPATH, '//*[@id="description"]/p').text.replace("\n", " ")}
    except: Descripcion = {"Descripción":None}
    print(Descripcion)

    try: Titulo1 = {"Titulo1":driver.find_element(By.XPATH, '//*[@id="main_info"]/h1').text}
    except: Titulo1 = {"Titulo1":driver.find_element(By.XPATH, '//*[@class="ap_traffic"]/h1').text}
    print(Titulo1)
    
    try:
        info_servicios = driver.find_elements(By.XPATH, '//*[@id="property_facilities"]/ul/li')
        servicios = []
        for item in info_servicios:
            servicios.append(item.text)
    
        if servicios == []:
            Servicios = {"Servicios": None}
        else: Servicios = {"Servicios": servicios}
    except: Servicios = {"Servicios": None}
    print(Servicios)

    try:
        info_servicios = driver.find_elements(By.XPATH, '//*[@id="building_facilities"]/ul/li')
        servicios = []
        for item in info_servicios:
            servicios.append(item.text)
        
        if servicios == []:
            Servicios_prop = {"Servicios_prop": None}
        else: Servicios_prop = {"Servicios_prop": servicios}
    except: Servicios_prop = {"Servicios_prop": None}
    print(Servicios_prop)

    
    #info_anfitrion = driver.find_element(By.XPATH, '//div[@class="contact-form-agency-info"]/*/p[@class="agency-name"]').text.split("\n")
    try:
        Anfitrion = {"Anfitrion": driver.find_elements(By.XPATH, '//div[@class="contact-form-agency-info"]/*/p[@class="agency-name"]')[1].text}
    except: Anfitrion = {"Anfitrion": None}
    print(Anfitrion)

    #Tipo_Anfitrion = {"Tipo_Anfitrion":info_anfitrion[1]}
    #print(Tipo_Anfitrion)
    
    #Registro = {"Registro":driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/aside/div[3]/div/div/div[2]/p[3]/strong').text}
    #print(Registro)

    try: Telefono = {"Telefono": driver.find_element(By.XPATH, '//*[@id="contact-form"]/div[2]/div[1]/div[@class="contact-form-chat-buttons"]/ul/li').get_attribute("data-contact")}
    except: Telefono = {"Telefono": None}
    print(Telefono)

    try: 
        info_direccion = driver.find_element(By.XPATH, '//*[@id="map_section_container"]/div[1]/p').text.split("\n")
        if len(info_direccion) > 1:
            Direccion = {"Direccion": info_direccion[0].split(",")[0]}
        else: Direccion = {"Direccion": None}
    except: Direccion = {"Direccion": None}
    print(Direccion)
    
    #Ubicacion = {"Ubicacion":driver.find_element(By.XPATH, '//div[@class="parairnos-google-maps"]/iframe').get_attribute("src")}
    #print(Ubicacion)

    try:
        info_ubicacion = driver.find_element(By.XPATH, '//*[@id="map"]')
        lat = {"long": info_ubicacion.get_attribute("data-latitude")}
        long =  {"long": info_ubicacion.get_attribute("data-longitude")}
    except: 
        lat = {"long": None}
        long =  {"long": None}
    print(lat, long)

    #n_free_date = {"n_free_date": driver.find_element(By.XPATH, '//div[contains(text(),"Llegada")]/../div[contains(@data-testid, "change-dates")]').text}
    
    try: Superficie = {"Superficie":driver.find_element(By.XPATH, '//div[contains(text(), "Superficie")]/../div[@class="amenity-value"]').text}
    except: Superficie = {"Superficie": None}

    try: Superficie_terreno = {"Superficie_terreno":driver.find_element(By.XPATH, '//div[contains(text(), "Superficie del terreno")]/../div[@class="amenity-value"]').text}
    except: Superficie_terreno = {"Superficie_terreno": None}

    Tipo = {"Tipo":driver.find_element(By.XPATH, '//div[contains(text(), "Tipo de propiedad")]/../div[@class="amenity-value"]').text}
    
    Tipo_anuncio = {"Tipo_anuncio":driver.find_element(By.XPATH, '//div[contains(text(), "Tipo de anuncio")]/../div[@class="amenity-value"]').text}

    try: Amueblado = {"Amueblado":driver.find_element(By.XPATH, '//div[contains(text(), "Amueblado")]/../div[@class="amenity-value"]').text}
    except: Amueblado = {"Amueblado":None}
    print(Amueblado)

    try: Piso = {"Piso":driver.find_element(By.XPATH, '//div[contains(text(), "Piso")]/../div[@class="amenity-value"]').text}
    except: Piso = {"Piso": None}

    try: Año_construccion = {"Año_construccion":driver.find_element(By.XPATH, '//div[contains(text(), "Año de construcción")]/../div[@class="amenity-value"]').text}
    except: Año_construccion = {"Año_construcion": None}

    ScrapDate = {"ScrapDate": datetime.now().date().strftime("%d-%m-%Y")}

    Fuente = {"Fuente": "Trovit"}
    
    dicts = [Link, Region, Baños, Habitaciones, Precio, Tipo_precio, Superficie, Superficie_terreno, Tipo, Tipo_anuncio, Amueblado,
        Descripcion, Titulo1, Servicios, Servicios_prop, Anfitrion, Telefono, lat, long, Piso, Año_construccion, ScrapDate, Fuente]
    
    #print(dicts)
    for dic in dicts:
        Comuna.update(dic)

    dfmin = pd.DataFrame([Comuna])
    dfpage = pd.concat([dfpage, dfmin])

    df = pd.concat([df,dfmin])
    return df, dfpage

#df = scrap_trovit(driver, link, df, dfpage)
#df

In [20]:
df = pd.DataFrame()

#link = "https://www.toctoc.com/propiedades/arriendocorredorasr/departamento/coquimbo/av-pedro-nolasco-videla-810-1664-1/2343459?utm_source=Lifull-connect&utm_medium=cpc&utm_campaign=lifull-connect-rent"
#link= "https://www.toctoc.com/propiedades/arriendocorredorasr/departamento/coquimbo/an-penuelas-norte-186/1974247#partialTabsUbicacion"
#link = "https://www.toctoc.com/propiedades/arriendocorredorasr/terrenoindustrial/coquimbo/barrio-industrial-chanar/1556328?utm_source=Lifull-connect&utm_medium=cpc&utm_campaign=lifull-connect-rent"
link = "https://www.toctoc.com/propiedades/arriendocorredorasr/parcela/coquimbo/se-arrienda-para-eventos-nocturnos-y-diurnos/2205342?utm_source=Lifull-connect&utm_medium=cpc&utm_campaign=lifull-connect-rent"
dfpage = pd.DataFrame()

def scrap_toctoc(driver, link, df, dfpage):
    print(link)
    #driver.get("https://www.toctoc.com/propiedades/arriendocorredorasr/terrenoindustrial/coquimbo/sector-el-panul/2303144?utm_source=Lifull-connect&utm_medium=cpc&utm_campaign=lifull-connect-rent")
    time.sleep(3)
    try:
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_all_elements_located((By.XPATH, '//*[@id="mapFichaUbicacion"]')))
        #time.sleep(3)
        googlemap = driver.find_element(By.XPATH, '//*[@id="mapFichaUbicacion"]')
        time.sleep(1)
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", googlemap)
        #time.sleep(4)
        #wait = WebDriverWait(driver, 10)
        #wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        link = driver.current_url

        Link = {"Link":link}
        print(Link)

        info_comuna = driver.find_element(By.XPATH, '//*[@id="partialCabecera"]/div[1]/header/div/h2')
        #print("info_comuna: ", info_comuna.text)
        comuna = info_comuna.text.replace(" Ver ubicación","").split(" ")
    
        Comuna = {"Comuna": comuna[-3]}
        print(Comuna)

        Region = {"Region": comuna[-1]}
        print(Region)

        Baños = {"Baños": driver.find_element(By.XPATH, '//ul[@class="info_ficha"]/li[@class="baños"]/strong').text}
        print(Baños)

        #Capacidad = {"Capacidad": max_number}
        #print(Capacidad)
        
        Habitaciones = {"Habitaciones": ''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//li[@class="dormitorios"]/strong').text))}
        print(Habitaciones)

        infoprecio = ''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//*[@id="partialCabecera"]/div[1]/div/div/div/strong').text))
        if infoprecio == "":    
            infoprecio = None
        else: Precio = {"Precio": infoprecio}
        print(Precio)

        if "$" in driver.find_element(By.XPATH, '//*[@id="partialCabecera"]/div[1]/div/div/div/strong').text:
            Tipo_precio = {"Tipo_precio": "CLP"}
        elif "UF" in driver.find_element(By.XPATH, '//*[@id="partialCabecera"]/div[1]/div/div/div/strong').text:
            Tipo_precio = {"Tipo_precio": "UF"}
        else: Tipo_precio = None
        print(Tipo_precio)
        
        #try: Nota = {"Nota":driver.find_element(By.XPATH, '//*[@id="userComments"]/div[1]/div/div/div[2]/div[1]').text}
        #except: Nota = {"Nota": None}
        #print(Nota)

        #try: Nnota = {"Nnota":''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//*[@id="userComments"]/div[1]/div/div/div[2]/div[3]').text))}
        #except: Nnota = {"Nota": None}
        #print(Nnota)

        #try: 
        #    info_comentarios = driver.find_elements(By.XPATH, '//*[@id="userComments"]/div[2]/div/div/div/div/p')
        #    comentarios = []
        #    for item in info_comentarios:
        #        comentarios.append(item.text)
        #    Comentarios = {"Comentarios": comentarios}
        #except: {"Comentarios": None}
        #print(Comentarios)

        try: Descripcion = {"Descripción":driver.find_element(By.XPATH, '//*[@id="partialInformacionAdicional"]/div[1]/div[2]').text.replace("\n", " ")}
        except: Descripcion = {"Descripción":None}
        print(Descripcion)

        Titulo1 = {"Titulo1":driver.find_element(By.XPATH, '//*[@id="partialCabecera"]/div[1]/header/h1').text}
        print(Titulo1)
        
        #info_servicios = driver.find_elements(By.XPATH, '//*[@id="property_facilities"]/ul/li')
        #servicios = []
        #for item in info_servicios:
        #    servicios.append(item.text)
        
        #if servicios == []:
        #    Servicios = {"Servicios": None}
        #else: Servicios = {"Servicios": servicios}
        #print(Servicios)

        #info_servicios = driver.find_elements(By.XPATH, '//*[@id="building_facilities"]/ul/li')
        #servicios = []
        #for item in info_servicios:
        #    servicios.append(item.text)
        
        #if servicios == []:
        #    Servicios_prop = {"Servicios_prop": None}
        #else: Servicios_prop = {"Servicios_prop": servicios}
        #print(Servicios_prop)

        
        #info_anfitrion = driver.find_element(By.XPATH, '//div[@class="contact-form-agency-info"]/*/p[@class="agency-name"]').text.split("\n")
        anfi_button = driver.find_element(By.XPATH, '//*[@id="btnVerDatosContacto"]')
        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", anfi_button)
        #time.sleep(1)
        #print(anfi_button.is_displayed())
        #print(anfi_button.is_enabled())
        anfi_button.click()
        time.sleep(1)
        Anfitrion = {"Anfitrion": driver.find_element(By.XPATH, '//*[@id="partialInformacionContacto"]/div/div[2]/ul/li[3]/div/h5/strong').text}
        print(Anfitrion)

        #Tipo_Anfitrion = {"Tipo_Anfitrion":info_anfitrion[1]}
        #print(Tipo_Anfitrion)
        
        #Registro = {"Registro":driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/aside/div[3]/div/div/div[2]/p[3]/strong').text}
        #print(Registro)

        try: Telefono = {"Telefono": ''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//*[@id="partialInformacionContacto"]/div/div[2]/ul/li[3]/div/p[2]').text))}
        except: Telefono = {"Telefono": None}
        print(Telefono)

        #info_direccion = driver.find_element(By.XPATH, '//*[@id="map_section_container"]/div[1]/p').text.split("\n")
        #if len(info_direccion) > 1:
        #    Direccion = {"Direccion": info_direccion[0].split(",")[0]}
        #else: Direccion = {"Direccion": None}
        #print(Direccion)
        
        #Ubicacion = {"Ubicacion":driver.find_element(By.XPATH, '//div[@class="parairnos-google-maps"]/iframe').get_attribute("src")}
        #print(Ubicacion)

        #info_ubicacion = driver.find_element(By.XPATH, '//*[@id="h_br_lat"]')
        lat = {"long": driver.find_element(By.XPATH, '//*[@id="h_br_lat"]').get_attribute("value")}
        long =  {"long": driver.find_element(By.XPATH, '//*[@id="h_br_lon"]').get_attribute("value")}
        print(lat, long)

        #n_free_date = {"n_free_date": driver.find_element(By.XPATH, '//div[contains(text(),"Llegada")]/../div[contains(@data-testid, "change-dates")]').text}
        
        try: Superficie = {"Superficie":  str(max(map(int, re.findall(r'\d+', driver.find_element(By.XPATH, '//*[@id="partialCabecera"]/div[1]/header/ul/li[1]').text))))}
        except: Superficie = {"Superficie": None}
        print(Superficie)

        try: Superficie_construida = {"Superficie_construida":''.join(filter(lambda x: x.isdigit() and x != '²', driver.find_element(By.XPATH, '//ul[@class="info_ficha"]/li[@class="metrosConstruidos"]/strong').text))}
        except: Superficie_construida = {"Superficie_construida": None}
        try: Superficie_util = {"Superficie_util":''.join(filter(lambda x: x.isdigit() and x != '²', driver.find_element(By.XPATH, '//ul[@class="info_ficha"]/li[@class="metrosUtiles"]/strong').text))}
        except: Superficie_util = {"Superficie_util": None}
        try: Superficie_terreno = {"Superficie_terreno":''.join(filter(lambda x: x.isdigit() and x != '²', driver.find_element(By.XPATH, '//ul[@class="info_ficha"]/li[@class="metrosTerreno"]/strong').text))}
        except: Superficie_terreno = {"Superficie_terreno": None}
        print(Superficie_construida, Superficie_terreno, Superficie_util)

        info_Tipo = driver.find_element(By.XPATH, '//*[@id="partialCabecera"]/div[1]/header/div/h2').text.split(" ")
        Tipo = {"Tipo": info_Tipo[0]}
        
        if info_Tipo[0] == "Terreno":
            Tipo = {"Tipo": info_Tipo[0]+" "+info_Tipo[1]}
        print(Tipo)

        info_anuncio = driver.find_element(By.XPATH, '//*[@id="partialCabecera"]/div[1]/header/div/h2').text.split(" ")
        Tipo_anuncio = {"Tipo_anuncio":info_anuncio[2]}
        
        if info_anuncio[2] == "en":
            Tipo_anuncio = {"Tipo_anuncio":info_anuncio[3]}
        print(Tipo_anuncio)

        #try: Amueblado = {"Amueblado":driver.find_element(By.XPATH, '//div[contains(text(), "Amueblado")]/../div[@class="amenity-value"]').text}
        #except: Amueblado = {"Amueblado":None}
        #print(Amueblado)

        #try: Piso = {"Piso":driver.find_element(By.XPATH, '//div[contains(text(), "Piso")]/../div[@class="amenity-value"]').text}
        #except: Piso = {"Piso": None}

        try: Año_construccion = {"Año_construccion": driver.find_element(By.XPATH, '//ul[@class="info_ficha"]/li[@class="año"]/strong').text}
        except: Año_construccion = {"Año_construcion": ""}
        print(Año_construccion)

        Superficie_terraza = {"Superficie_terraza":''.join(filter(lambda x: x.isdigit() and x != '²', driver.find_element(By.XPATH, '//ul[@class="info_ficha"]/li[@class="metrosTerraza"]/strong').text))}
        estacionamientos = {"Estacionamientos":driver.find_element(By.XPATH, '//ul[@class="info_ficha"]/li[@class="estacionamientos"]/strong').text}

        driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", driver.find_element(By.XPATH, '//*[@id="infoEntorno"]'))
        time.sleep(1)
        try:
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", driver.find_element(By.XPATH, '//*[@id="headingEntorno"]/h4[contains(text(),"verdes")]'))
            time.sleep(1)
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", driver.find_element(By.XPATH, '//*[@id="headingEntorno"]/h4[contains(text(),"Seguridad")]'))

            infoind = driver.find_elements(By.XPATH,'//*[@id="entornoContenidoSeguridad"]/div/div[2]/div[1]/div/span')
            for span in infoind:
                if span.find_elements(By.XPATH, './/small'):
                    ind = span.get_attribute('class')
            ind_delinc = {"Ind_delinc": 6-int(''.join(filter(str.isdigit, ind)))}

            infoind = driver.find_elements(By.XPATH,'//*[@id="entornoContenidoSeguridad"]/div/div[2]/div[2]/div/span')
            for span in infoind:
                if span.find_elements(By.XPATH, './/small'):
                    ind = span.get_attribute('class')
            ind_seguridad = {"Ind_seguridad": ''.join(filter(str.isdigit, ind))}
            
            infoind = driver.find_elements(By.XPATH,'//*[@id="entornoContenidoTransporte"]/div/div[2]/div[2]/div/span')
            for span in infoind:
                if span.find_elements(By.XPATH, './/small'):
                    ind = span.get_attribute('class')
            ind_transporte = {"Ind_transporte": ''.join(filter(str.isdigit, ind))}
        
            Averdexpersona = {"Averdexpersona": ''.join(filter(lambda x: x.isdigit() and x != '²', driver.find_element(By.XPATH, '//*[@id="entornoContenidoAreaVerde"]/div/div[2]/div[2]/p/strong').text))}
            
        except:
            Averdexpersona = {"Averdexpersona": None}
            ind_transporte = {"Ind_transporte": None}
            ind_delinc = {"Ind_delinc": None}
            ind_seguridad = {"Ind_seguridad": None}

        print(ind_delinc)
        print(ind_seguridad)
        print(ind_transporte)
        print(Averdexpersona)
        ScrapDate = {"ScrapDate": datetime.now().date().strftime("%d-%m-%Y")}
        print(ScrapDate)
        Fuente = {"Fuente": "Toctoc"}
        print(Fuente)
        
        dicts = [Link, Titulo1, Region, Baños, Habitaciones, Precio, Tipo_precio, Superficie, Superficie_terreno, Superficie_construida, Superficie_terraza, Tipo, Tipo_anuncio, estacionamientos,
            ind_delinc, ind_seguridad, ind_transporte, Averdexpersona, Descripcion,  Anfitrion, Telefono, lat, long, Año_construccion, ScrapDate, Fuente]
        
        #print(dicts)
        for dic in dicts:
            Comuna.update(dic)

        dfmin = pd.DataFrame([Comuna]).replace("", None)
        dfpage = pd.concat([dfpage, dfmin])

        df = pd.concat([df,dfmin])
        return df, dfpage
    except: 
        try: 
            driver.find_element(By.XPATH, '//*[@id="modalIngresoRegistro"]//button[@aria-label="Close"]').click()
            df, dfpage = scrap_toctoc(driver, link, df, dfpage)
            return df, dfpage
        except:
                print("LINK PERDIDO")
                return df, dfpage

df = scrap_toctoc(driver, link, df, dfpage)
df

{'Link': 'https://www.toctoc.com/propiedades/arriendocorredorasr/terrenoindustrial/coquimbo/sector-el-panul/2303144?utm_source=Lifull-connect&utm_medium=cpc&utm_campaign=lifull-connect-rent'}
{'Comuna': 'Coquimbo'}
{'Region': 'Coquimbo'}
{'Baños': ''}
{'Habitaciones': ''}
{'Precio': '100'}
{'Tipo_precio': 'UF'}
{'Descripción': 'Terreno Industrial orilla carretera, ubicado sector Panul, cierre perimetral con bulldog. Usos Permitidos: Industrial Inofensivo / Molesto, Almacenamiento.  .'}
{'Titulo1': 'SECTOR EL PANUL'}
{'Anfitrion': ''}
{'Telefono': ''}
{'long': '-29.9985'} {'long': '-71.3731'}
{'Superficie': '20000'}
{'Superficie_construida': '20000'} {'Superficie_terreno': ''} {'Superficie_util': ''}
{'Tipo': 'Terreno Industrial'}
{'Tipo_anuncio': 'Arriendo'}
{'Año_construccion': ''}
{'Ind_delinc': None}
{'Ind_seguridad': None}
{'Ind_transporte': None}
{'Averdexpersona': None}
{'ScrapDate': '21-02-2023'}
{'Fuente': 'Toctoc'}


(     Comuna                                               Link  \
 0  Coquimbo  https://www.toctoc.com/propiedades/arriendocor...   
 
            Titulo1    Region Baños Habitaciones Precio Tipo_precio Superficie  \
 0  SECTOR EL PANUL  Coquimbo  None         None    100          UF      20000   
 
   Superficie_terreno  ... Ind_seguridad Ind_transporte Averdexpersona  \
 0               None  ...          None           None           None   
 
                                          Descripción Anfitrion Telefono  \
 0  Terreno Industrial orilla carretera, ubicado s...      None     None   
 
        long Año_construccion   ScrapDate  Fuente  
 0  -71.3731             None  21-02-2023  Toctoc  
 
 [1 rows x 26 columns],
      Comuna                                               Link  \
 0  Coquimbo  https://www.toctoc.com/propiedades/arriendocor...   
 
            Titulo1    Region Baños Habitaciones Precio Tipo_precio Superficie  \
 0  SECTOR EL PANUL  Coquimbo  None         No

In [7]:
df = pd.DataFrame()
link = "https://cl.tixuz.com/inmuebles/renta/departamento/arriendo-bosque-san-carlos--coquimbo/16949992?utm_source=Trovit&utm_medium=CPC&utm_campaign=Chile-Casas"
link = "https://cl.tixuz.com/inmuebles/renta/departamento/departamento-en-la-herradura--coquimbo/16829774?utm_source=Trovit&utm_medium=CPC&utm_campaign=Chile-Casas"
link = "https://cl.tixuz.com/inmuebles/renta/departamento/departamento-en-la-herradura--coquimbo/16829774?utm_source=Trovit&utm_medium=CPC&utm_campaign=Chile-Casas"
link = "https://cl.tixuz.com/inmuebles/renta/departamento/marzo-a-diciembre--la-herradura--coquimbo/17069546?utm_source=Trovit&utm_medium=CPC&utm_campaign=Chile-Casas"
link = "https://cl.tixuz.com/inmuebles/renta/departamento/arriendo-departamento-en-coquimbo-excelente-ubicaci%c3%b3n/17000356?utm_source=Trovit&utm_medium=CPC&utm_campaign=Chile-Casas"
link = "https://cl.tixuz.com/inmuebles/renta/departamento/departamento-el-mirador-de-coquimbo-coquimbo/16965845?utm_source=Trovit&utm_medium=CPC&utm_campaign=Chile-Casas"
dfpage = pd.DataFrame()

def scrap_tixuz(driver, link, df, dfpage):
    #driver.get(link)
    time.sleep(3)
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.XPATH, '//p[@itemprop="description"]')))
    #time.sleep(3)
    googlemap = driver.find_element(By.XPATH, '//p[@itemprop="description"]')
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", googlemap)
    #time.sleep(4)
    #wait = WebDriverWait(driver, 10)
    #wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    link = driver.current_url

    Link = {"Link":link}
    print(Link)
    #info_comuna = driver.find_element(By.XPATH, '//*[@id="main_info"]/h2').text
    #comuna = info_comuna.split(", ")

    #if len(comuna) > 3:
    #    Comuna = {"Comuna": comuna[1]}
    #else:   
    #    Comuna = {"Comuna": comuna[0]}
    #print(Comuna)

    Region = {"Region": driver.find_element(By.XPATH, '//span[@itemprop="addressRegion"]').text}
    print(Region)

    Baños = {"Baños": ''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//div[@class="composs-main-article-meta"][2]/*[contains(text(), "año")]').text))}
    print(Baños)

    #Capacidad = {"Capacidad": max_number}
    #print(Capacidad)
    
    Habitaciones = {"Habitaciones": ''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//div[@class="composs-main-article-meta"][2]/*[contains(text(), "camara")]').text))}
    print(Habitaciones)

    infoprecio = ''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//span[@itemprop="price"]').text))
    if infoprecio == "":    
        infoprecio = None
    else: Precio = {"Precio": infoprecio}
    print(Precio)

    if "CLP" in driver.find_element(By.XPATH, '//span[@itemprop="priceCurrency"]').text:
        Tipo_precio = {"Tipo_precio": "CLP"}
    elif "UF" in driver.find_element(By.XPATH, '//span[@itemprop="priceCurrency"]').text:
        Tipo_precio = {"Tipo_precio": "UF"}
    else: Tipo_precio = {"Tipo_precio": None}
    print(Tipo_precio)
    
    #try: Nota = {"Nota":driver.find_element(By.XPATH, '//*[@id="userComments"]/div[1]/div/div/div[2]/div[1]').text}
    #except: Nota = {"Nota": None}
    #print(Nota)

    #try: Nnota = {"Nnota":''.join(filter(str.isdigit, driver.find_element(By.XPATH, '//*[@id="userComments"]/div[1]/div/div/div[2]/div[3]').text))}
    #except: Nnota = {"Nota": None}
    #print(Nnota)

    #try: 
    #    info_comentarios = driver.find_elements(By.XPATH, '//*[@id="userComments"]/div[2]/div/div/div/div/p')
    #    comentarios = []
    #    for item in info_comentarios:
    #        comentarios.append(item.text)
    #    Comentarios = {"Comentarios": comentarios}
    #except: {"Comentarios": None}
    #print(Comentarios)

    try: Descripcion = {"Descripción":driver.find_element(By.XPATH, '//p[@itemprop="description"]').text.replace("\n", " ")}
    except: Descripcion = {"Descripción":None}
    print(Descripcion)

    Titulo1 = {"Titulo1":driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div/div/div/div/div[1]/div/h1').text}
    print(Titulo1)
    
    #info_servicios = driver.find_elements(By.XPATH, '//*[@id="property_facilities"]/ul/li')
    #servicios = []
    #for item in info_servicios:
    #    servicios.append(item.text)
    
    #if servicios == []:
    #    Servicios = {"Servicios": None}
    #else: Servicios = {"Servicios": servicios}
    #print(Servicios)
    
    #info_anfitrion = driver.find_element(By.XPATH, '//div[@class="contact-form-agency-info"]/*/p[@class="agency-name"]').text.split("\n")
    #Anfitrion = {"Anfitrion": driver.find_elements(By.XPATH, '//div[@class="contact-form-agency-info"]/*/p[@class="agency-name"]')[1].text}
    #print(Anfitrion)
    
    #Registro = {"Registro":driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[2]/aside/div[3]/div/div/div[2]/p[3]/strong').text}
    #print(Registro)

    #try: Telefono = {"Telefono": driver.find_element(By.XPATH, '//*[@id="contact-form"]/div[2]/div[1]/div[@class="contact-form-chat-buttons"]/ul/li').get_attribute("data-contact")}
    #except: Telefono = {"Telefono": None}
    #print(Telefono)

    #info_direccion = driver.find_element(By.XPATH, '//*[@id="map_section_container"]/div[1]/p').text.split("\n")
    #if len(info_direccion) > 1:
    #    Direccion = {"Direccion": info_direccion[0].split(",")[0]}
    #else: Direccion = {"Direccion": None}
    #print(Direccion)
    
    #Ubicacion = {"Ubicacion":driver.find_element(By.XPATH, '//div[@class="parairnos-google-maps"]/iframe').get_attribute("src")}
    #print(Ubicacion)

    #info_ubicacion = driver.find_element(By.XPATH, '//*[@id="map"]')
    #lat = {"long": info_ubicacion.get_attribute("data-latitude")}
    #long =  {"long": info_ubicacion.get_attribute("data-longitude")}
    #print(lat, long)

    #n_free_date = {"n_free_date": driver.find_element(By.XPATH, '//div[contains(text(),"Llegada")]/../div[contains(@data-testid, "change-dates")]').text}
    
    #try: Superficie = {"Superficie":driver.find_element(By.XPATH, '//div[contains(text(), "Superficie")]/../div[@class="amenity-value"]').text}
    #except: Superficie = {"Superficie": None}

    try: 
        Superficie = {"Superficie":''.join(filter(lambda x: x.isdigit(),
            driver.find_element(By.XPATH, '//div[@class="composs-main-article-meta"][2]//*[contains(text(), "m2")]/..').text.replace("m2","")))}
    except: Superficie = {"Superficie": None}
    print(Superficie)

    try:
        Estacionamientos = {"Estacionamientos":driver.find_element(By.XPATH, '//div[@class="composs-main-article-meta"][2]//*[contains(text(), "Estacion")]')}
    except: Estacionamientos = {"Estacionamientos":None}

    #Tipo = {"Tipo":driver.find_element(By.XPATH, '//div[contains(text(), "Tipo de propiedad")]/../div[@class="amenity-value"]').text}
    
    #Tipo_anuncio = {"Tipo_anuncio":driver.find_element(By.XPATH, '//div[contains(text(), "Tipo de anuncio")]/../div[@class="amenity-value"]').text}

    #try: Amueblado = {"Amueblado":driver.find_element(By.XPATH, '//div[contains(text(), "Amueblado")]/../div[@class="amenity-value"]').text}
    #except: Amueblado = {"Amueblado":None}
    #print(Amueblado)

    #try: Piso = {"Piso":driver.find_element(By.XPATH, '//div[contains(text(), "Piso")]/../div[@class="amenity-value"]').text}
    #except: Piso = {"Piso": None}

    #try: Año_construccion = {"Año_construccion":driver.find_element(By.XPATH, '//div[contains(text(), "Año de construcción")]/../div[@class="amenity-value"]').text}
    #except: Año_construccion = {"Año_construcion": None}

    ScrapDate = {"ScrapDate": datetime.now().date().strftime("%d-%m-%Y")}

    Fuente = {"Fuente": "Tixuz"}
    
    dicts = [Link, Region, Baños, Habitaciones, Precio, Tipo_precio, Superficie, 
        Descripcion, Titulo1, Estacionamientos, ScrapDate, Fuente]
    
    #print(dicts)
    concat = {}
    for dic in dicts:
        concat.update(dic)

    dfmin = pd.DataFrame([concat])
    dfpage = pd.concat([dfpage, dfmin])

    df = pd.concat([df,dfmin])
    return df, dfpage

#df = scrap_tixuz(driver, link, df, dfpage)
#df

In [8]:
df = pd.DataFrame()
pag = 1
def scrap_links(driver, df, pag):
    initial_url = driver.current_url
    #print(type(initial_url))
    #print(initial_url)
    items = driver.find_elements(By.XPATH, '//*[@id="wrapper_listing"]/li/div/div/div[2]/a')
    print(len(items), f"ITEMS ENCONTRADOS EN {pag}")
    links = []
    dfpage = pd.DataFrame()
    for item in items:
        #print(item.get_attribute("href"))
        links.append(item.get_attribute("href"))

    for index, link in enumerate(links):
        #print(link)
        print("debug1")
        driver.get(link)
        print("debug2")
        time.sleep(3)
        print("debug3")

        stringurl = driver.current_url
        print("cargando...", index+1)
        if "tixuz" in stringurl[:25]:
            print("tixuz")
            df, dfpage = scrap_tixuz(driver, link, df, dfpage)
        elif "toctoc" in stringurl[:25]:
            print("toctoc")
            df, dfpage = scrap_toctoc(driver, link, df, dfpage)
        elif "trovit" in stringurl[:25]:
            print("trovit")
            df, dfpage = scrap_trovit(driver, link, df, dfpage)
        else: print("NO CARGA/NO SE RECONOCE")

        #df, dfpage = scrap(driver, link, df, dfpage)

    driver.get(initial_url)
    dfpage.to_csv(os.path.join(folderpath, f"Data_{pag}"))
    pag = pag + 1
    return df, pag

df, pag = scrap_links(driver, df, pag)

30 ITEMS ENCONTRADOS EN 1
debug1
debug2
debug3
cargando... 1
trovit
{'Link': 'https://casas.trovit.cl/listing/arriendo-amoblado-palmas-de-penuelas-coquimbo.aab5dfeb-6749-428f-aba6-151fd337b5b0'}
{'Comuna': 'Coquimbo'}
{'Region': 'Coquimbo'}
{'Baños': None}
{'Habitaciones': '3'}
{'Precio': '500000'}
{'Tipo_precio': 'CLP'}
{'Descripción': 'Se arrienda hermoso departamento en condominio Palmas de Peñuelas, ubicado en Coquimbo🎉  Propiedad en 8° piso 🏢  🌊 Amplio living comedor.  🌊 Amplia cocina equipada y amoblada , con logia.  🌊 3 Dormitorios :  Dormitorio principal en suite.  2 ° y 3° dormitorio de buen tamaño.  🌊 2 baños.  🌊 Terraza.  🌊 Estacionamiento en superficie.  🌊 Bodega  🌊 Condominio con quincho, aéreas verdes, juegos infantiles y piscina.  🌊 Conserjería las 24 horas.  🌊 Departamento amoblado 🌊  Gran Oportunidad $500000 con gasto común incluido 🎇'}
{'Titulo1': 'ARRIENDO AMOBLADO PALMAS DE PEÑUELAS COQUIMBO'}
{'Servicios': ['Balcón', 'Armarios empotrados', 'Estacionamiento', 'Elect

In [15]:
def siguiente_pagina(driver):
    try: 
        button = driver.find_element(By.XPATH, '//p[@class="trovit-paginator"]/a[contains(text(),"Siguiente")]')
        print("Cargando siguiente página    ----------------------------------------------------------------------------")
        #time.sleep(5)
        #print("Link CARGANDO", button.get_attribute("href"))
        #if button.is_enabled():
        button.click()
        y = 0
        return y
    except: 
        print("NO HAY SIGUIENTE PÁGINA")
        y = 1
        return y
y = siguiente_pagina(driver)

NO HAY SIGUIENTE PÁGINA


In [17]:
print(folderpath)

C:\Users\bigdatauls\Documents\workspace\python\Webscrapper_arriendos\Trovit\Data_porpag_1


In [18]:
df, pag = scrap_links(driver, df, pag)
y = siguiente_pagina(driver)
df, pag = scrap_links(driver, df, pag)
y = siguiente_pagina(driver)
df, pag = scrap_links(driver, df, pag)
y = siguiente_pagina(driver)

30 ITEMS ENCONTRADOS EN 2
debug1
debug2
debug3
cargando... 1
toctoc
https://rd.clk.thribee.com/id.Sv1J1o1M1J1U1C/origin.2/section.1/section_type.1/country.cl/vertical.homes/iuid.30240web-modphp-1563f4d5fa7c55e/tracking/what_d.Coquimbo%20Region/type.2/page.2/cookieid.1468323716263f4c9be9ba460.22241644/campaigns.1462/pos.1/
{'Link': 'https://www.toctoc.com/propiedades/arriendocorredorasr/terreno/coquimbo/la-portada-mz-1-st-21-coquimbo/1920154?utm_source=Lifull-connect&utm_medium=cpc&utm_campaign=lifull-connect-rent'}
{'Comuna': 'Coquimbo'}
{'Region': 'Coquimbo'}
{'Baños': ''}
{'Habitaciones': ''}
{'Precio': '95000'}
{'Tipo_precio': 'CLP'}
{'Descripción': 'Casa Vacacional 3 dormitorios 2 baños 4 ambientes cocina americana 3 estacionamientos a mts de la Playa'}
{'Titulo1': 'LA PORTADA MZ 1 ST 21, Coquimbo'}
{'Anfitrion': ''}
{'Telefono': ''}
{'long': '-30.195923'} {'long': '-71.396369'}
{'Superficie': '350'}
{'Superficie_construida': ''} {'Superficie_terreno': '350'} {'Superficie_util': ''